In [1]:
import pandas as pd
import zstandard
from tqdm import tqdm

<li><strong>C4</strong> (<img src="luisito.jpg"><img src="luisito.jpg"><img src="luisito.jpg">). Si tomamos una muestra de negocios con más de 10 reviews y todas sus reviews como un texto concatenado, en términos de distancia de compresión normalizada para esos textos, cuáles son los dos negocios más parecidos? Muestrelos sus textos y distancia.</li>

In [2]:
reviews = pd.read_csv("../review.csv", usecols=["business_id", "text"])
reviews.dropna(inplace=True)

In [3]:
business = pd.read_csv("../business.csv", usecols=["business_id", "name"])
business.dropna(inplace=True)

In [4]:
merged = business.merge(reviews, left_on="business_id", right_on="business_id")

In [5]:
merged

,business_id,name,text
0,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,I went in to ship a package to my friend for h...
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,I always go to this UPS location. The store is...
2,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,"If you really want to meet mean people, you sh..."
3,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,I've been to this UPS store several times now....
4,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,Will not answer the phone. Impossible to talk ...
...,...,...,...
3838100,jV_XOycEzSlTx-65W906pg,Sic Ink,It is with my greatest pleasure to say that I'...
3838101,jV_XOycEzSlTx-65W906pg,Sic Ink,I went there to get my nose piercing replaced....
3838102,jV_XOycEzSlTx-65W906pg,Sic Ink,Fantastic tattoo artist. Friendly and talkativ...
3838103,jV_XOycEzSlTx-65W906pg,Sic Ink,"Please note, this studio has relocated to Apol..."


In [6]:
grouped = merged.groupby("business_id").agg({"business_id":"count"})

In [7]:
mayores = grouped[grouped["business_id"] >= 10]
mayores

,business_id
business_id,
---kPU91CF4Lq2-WlRu9Lw,24
--8IbOsAAxjKRoYsBFL-PA,17
--9osgUCSDUWUkoTLdvYhQ,19
--FcbSxK1AoEtEAxOgBaCw,42
--N9yp3ZWqQIm7DqKRvorg,14
...,...
zzfj1-iPfw0cwnOjY0yUgA,11
zzg-Il9zxsaVXlCDrcG7hg,15
zzjFdJwXuxBOGe9JeY_EMw,32


In [8]:
del mayores["business_id"]

In [9]:
mayores = merged[merged["business_id"].isin(mayores.index)]
mayores

,business_id,name,text
8,tUFrWirKiKi_TAnsVWINQQ,Target,We are fans of Target. They seem to have a li...
9,tUFrWirKiKi_TAnsVWINQQ,Target,When I feel like ditching Walmart for a more c...
10,tUFrWirKiKi_TAnsVWINQQ,Target,I used to frequent this store for toys and col...
11,tUFrWirKiKi_TAnsVWINQQ,Target,Was there today around 6pm\n\nHere is somethin...
12,tUFrWirKiKi_TAnsVWINQQ,Target,Our cashier Hanny was very kind to us and gave...
...,...,...,...
3838092,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,"Visited the week prior to Christmas, staff hel..."
3838093,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,We purchased our bikes (three) a couple of wee...
3838094,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,Went there today to pick up cleats for bike sh...
3838095,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,Stopped in today to take the big leap into cyc...


In [10]:
mayoresGrouped = mayores.groupby("business_id").agg({"text":"sum"})

In [11]:
mayoresGrouped.reset_index(inplace=True)
mayoresGrouped

,business_id,text
0,---kPU91CF4Lq2-WlRu9Lw,Ate here for the 1st time on Saturday 08/07/20...
1,--8IbOsAAxjKRoYsBFL-PA,"Took me for 40 bucks through grubhub, no call ..."
2,--9osgUCSDUWUkoTLdvYhQ,My husband and I played two of the escape room...
3,--FcbSxK1AoEtEAxOgBaCw,This is my go to car wash establishment. Today...
4,--N9yp3ZWqQIm7DqKRvorg,"Made an appointment for 8am, get there at 7:50..."
...,...,...
68478,zzfj1-iPfw0cwnOjY0yUgA,I recently stopped here for lunch.I had debs c...
68479,zzg-Il9zxsaVXlCDrcG7hg,"We went on a Sunday afternoon, and got the bes..."
68480,zzjFdJwXuxBOGe9JeY_EMw,OMG this place is the shit. Like it's not only...
68481,zznJox6-nmXlGYNWgTDwQQ,I ordered two donuts and an egg sandwich. They...


In [12]:
mayoresGrouped["text"][0]

'Ate here for the 1st time on Saturday 08/07/2020. \nThe food was ok, had the Lobster Boil . The Lobster was ok for a soft shelled Lobster. The Crabs were soggy and mushy. The Clams were gummy and snotty . The headed shrimp were large and tasty. All the items were precooked, obviously. The fish spread was decent , also had the Fried Shrimp and Shrimp scampi  which were decent. BYOB ... ... The atmosphere was nice except for the zillion Flies. I\'ll be back with a six packAbsolutely fantastic!!!!  I\'ll be back for sure. Fried lobster tail was delicious. Five star for sure.I was sadly disappointed in my experience. We ordered crabs and they were extremely small, watery and over half were bad(sick crabs), we ordered beans and rice which lacked flavor and literally had one bean in it that you can see from the picture. The coleslaw also lacked taste. We ask for grouper nuggets and a few were not cooked all the way. So given the food experience it was nothing like the reviews we read. Why I

In [13]:
# sacadas de la catedra
def compress_size(bytefile):
    return len(zstandard.compress(bytefile))

def ncd(obj1, obj2):
    len1 = compress_size(obj1)
    len2 = compress_size(obj2)
    concat_len = len(zstandard.compress(obj1+obj2))
    return (concat_len-min(len1, len2))/max(len1, len2)

Lo primero que voy a hacer es aplicarle el compress a cada uno de los 68483 

In [14]:
mayoresGrouped.loc[:,"comp_size"] = mayoresGrouped["text"].apply(lambda x: compress_size(x.encode()))

Ahora lo que deberia hacer es a cada uno de los 68000 compararlos con los otros y quedarme con el NCD menor

In [15]:
mayoresGrouped

,business_id,text,comp_size
0,---kPU91CF4Lq2-WlRu9Lw,Ate here for the 1st time on Saturday 08/07/20...,3333
1,--8IbOsAAxjKRoYsBFL-PA,"Took me for 40 bucks through grubhub, no call ...",2624
2,--9osgUCSDUWUkoTLdvYhQ,My husband and I played two of the escape room...,2787
3,--FcbSxK1AoEtEAxOgBaCw,This is my go to car wash establishment. Today...,6997
4,--N9yp3ZWqQIm7DqKRvorg,"Made an appointment for 8am, get there at 7:50...",4594
...,...,...,...
68478,zzfj1-iPfw0cwnOjY0yUgA,I recently stopped here for lunch.I had debs c...,3172
68479,zzg-Il9zxsaVXlCDrcG7hg,"We went on a Sunday afternoon, and got the bes...",4086
68480,zzjFdJwXuxBOGe9JeY_EMw,OMG this place is the shit. Like it's not only...,4919
68481,zznJox6-nmXlGYNWgTDwQQ,I ordered two donuts and an egg sandwich. They...,3701


tomo, de ejemplo, 10 negocios y le calculo el ncd

In [16]:
selNegocios = mayoresGrouped.sample(10)
selNegocios

,business_id,text,comp_size
35040,VqBFrv6StMLgZmO_nCtk4A,Do they forget your items sometimes? Yep.\nDo ...,6794
68131,zdv6HmtViGmQn4PVk9bAWQ,I've been to this location a few times now! \n...,5132
49383,iIE1QV6JPA1yJ_hM_G725Q,Yuki did a beautiful job with my lash extensio...,20999
62365,uKnRBYzVk3jhrAoKoOY38w,The Lucky Anchor Pub is a local favorite. Grea...,3408
60329,sTo7vhHisNhhBkTP6rXCWw,"Limited Sandwich Choices, Expensive, Short lis...",17657
3067,1mozd2AVfh6erj3av0-52Q,My wife and I needed a quick getaway so we hea...,2137
17974,FlSiPPvYiqqDbKb8E_lLNg,I LOVE this location. When I am in STL visitin...,9840
9903,8A5LSwsKK5vXDH1M3_Tf1w,Not up to par with the average Ritz. it is far...,221986
45632,eoxP7LZmxINRAmXWhsq8ww,"Maybe they don't deserve 5 stars, but they sur...",3916
13545,BZls_LgW4GVRp9X5ngwgbQ,Very nice family-owned business. People who wo...,2581


In [17]:
#Hago esta funcion para ahorrarme calcular los comp
def ncdMod(pivot, pivot_Csize, text, text_Csize):
    len1 = pivot_Csize
    len2 = text_Csize
    concat_len = len(zstandard.compress(pivot+text))
    return (concat_len-min(len1, len2))/max(len1, len2)

In [18]:
ncdList = []
idList = []
for pivot_id, pivot, pivot_Csize in tqdm(selNegocios.values.tolist()):
    min_ncd=100
    for text_id,text, text_Csize in mayoresGrouped.values.tolist():
        if pivot_id == text_id: continue
        value = ncdMod(pivot.encode(),pivot_Csize,text.encode(),text_Csize)
        if value<min_ncd:
            min_ncd = value
            best_id = text_id
    ncdList.append(min_ncd)
    idList.append(best_id)

100%|██████████| 10/10 [08:42<00:00, 52.26s/it]


In [19]:
selNegocios.loc[:,"min_NCD"] = ncdList
selNegocios.loc[:,"similar_id"] = idList

In [20]:
selNegocios.sort_values(by="min_NCD", inplace=True)
selNegocios

,business_id,text,comp_size,min_NCD,similar_id
35040,VqBFrv6StMLgZmO_nCtk4A,Do they forget your items sometimes? Yep.\nDo ...,6794,0.873479,Tac9ES3xcFw-l0bChzQb4A
13545,BZls_LgW4GVRp9X5ngwgbQ,Very nice family-owned business. People who wo...,2581,0.875630,mlzUbkIgIvDOzHtA4hhhgA
3067,1mozd2AVfh6erj3av0-52Q,My wife and I needed a quick getaway so we hea...,2137,0.875994,7T8_htSMHuSzKqiwjSt3iA
68131,zdv6HmtViGmQn4PVk9bAWQ,I've been to this location a few times now! \n...,5132,0.877241,KH5-_-Mr20Y0AUXxk4VelQ
45632,eoxP7LZmxINRAmXWhsq8ww,"Maybe they don't deserve 5 stars, but they sur...",3916,0.884654,QF2MmFhwbkULKflpZc9N2A
62365,uKnRBYzVk3jhrAoKoOY38w,The Lucky Anchor Pub is a local favorite. Grea...,3408,0.892548,sQip0JS4H2ftKnKJqT4X0A
17974,FlSiPPvYiqqDbKb8E_lLNg,I LOVE this location. When I am in STL visitin...,9840,0.895325,LIMcWAe750ULlcO4SXBHDw
49383,iIE1QV6JPA1yJ_hM_G725Q,Yuki did a beautiful job with my lash extensio...,20999,0.919425,aKFs_M-L-kFJZ_mLBC35uA
60329,sTo7vhHisNhhBkTP6rXCWw,"Limited Sandwich Choices, Expensive, Short lis...",17657,0.922014,E0KwLVaQ-uKEjJl3j_VAGQ
9903,8A5LSwsKK5vXDH1M3_Tf1w,Not up to par with the average Ritz. it is far...,221986,0.960543,MWKowhzHQ_scqnJEqAUYgA


In [21]:
negocio1 = merged[merged["business_id"] == selNegocios[:1].business_id.item()]
negocio1.head()

,business_id,name,text
2075617,VqBFrv6StMLgZmO_nCtk4A,Nico's Taco Shop,Do they forget your items sometimes? Yep.\nDo ...
2075618,VqBFrv6StMLgZmO_nCtk4A,Nico's Taco Shop,"Was reluctant as there were mixed reviews, but..."
2075619,VqBFrv6StMLgZmO_nCtk4A,Nico's Taco Shop,This Nicos has a salsa bar which is nice. I en...
2075620,VqBFrv6StMLgZmO_nCtk4A,Nico's Taco Shop,The nacho supreme is the best ever with lots o...
2075621,VqBFrv6StMLgZmO_nCtk4A,Nico's Taco Shop,Horrible customer service. I ask for my carne ...


In [22]:
negocio1 = merged[merged["business_id"] == selNegocios[:1].similar_id.item()]
negocio1.head()

,business_id,name,text
2474404,Tac9ES3xcFw-l0bChzQb4A,Vallarta Mexican Restaurant,Came here with coworkers for a quick business ...
2474405,Tac9ES3xcFw-l0bChzQb4A,Vallarta Mexican Restaurant,20 minutes for nothing. Walked in. Hostess ask...
2474406,Tac9ES3xcFw-l0bChzQb4A,Vallarta Mexican Restaurant,"Came tonight for a light dinner, arriving at a..."
2474407,Tac9ES3xcFw-l0bChzQb4A,Vallarta Mexican Restaurant,This is probably my favorite place to come and...
2474408,Tac9ES3xcFw-l0bChzQb4A,Vallarta Mexican Restaurant,What a joke!!! Waited 1.5 hours more than the...


In [23]:
mayoresGrouped[mayoresGrouped["business_id"] == selNegocios[:1].business_id.item()].text.item()

'Do they forget your items sometimes? Yep.\nDo they forget to salt the beans sometimes? Yep\nBut in the end, you get what you get. This is Nicos. They all have their ups and downs. This place has always treated me and my significant other well. We enjoy it often.Was reluctant as there were mixed reviews, but decided to give them a try. I had the 3 rolled tacos ($3.19 really?  What a deal!) and they were deliciousThis Nicos has a salsa bar which is nice. I enjoyed my food as I do from their other locations. Chile relleno burrito, get in my belly!The nacho supreme is the best ever with lots of carne asada and no shortage of guacamole and sour cream. Plus I like their hot ads salsa to dress it with rather than glorified tomato paste.Horrible customer service. I ask for my carne asada cooked extra well done and cook just never gives a damn. I repeatedly ask them too and double check before they give me order but always fail. Never fail to mess up my order. I live 1 block away, but i will c

In [24]:
mayoresGrouped[mayoresGrouped["business_id"] == selNegocios[:1].similar_id.item()].text.item()

'Came here with coworkers for a quick business outing / meeting. It\'s Tuesday so we ordered the dollar tacos .. literally worst tacos I\'ve ever had . To be fair I haven\'t have anything else off the menu except the dollar tacos, but they were enough to deter me from returning . I\'m Mexican, I know good tacos and those were terrible ! Service was pretty bad too, I wouldn\'t recommend it20 minutes for nothing. Walked in. Hostess asked for 5 minutes while there obviously 10+ tables open for service. Sat down to some decent chips and salsa. No service for 5-10 minutes. I walked to the bar to figure out what was going on. Grabbed an extra salsa from the bar and asked the hostess what was the deal. She said she would get a waiter right away. He never showed up. We left 5 minutes later. Oh well.Came tonight for a light dinner, arriving at around 5. Only 1 other table taken in the entire restaurant and they sit us in a booth that was dirty and had seeds stuck to it. Not off to a good start.